# mergent

> Retrieve and process data from WRDS Mergent FISD

Since this is a proprietary dataset, in the documentation below, I can not show any of the data that is being retrieved/generated (will show only column names).

In [ ]:
#| default_exp wrds.mergent

In [ ]:
#|exports
from __future__ import annotations
from typing import List

import pandas as pd

from finsets.wrds import wrds_api

In [ ]:
#| exports
PROVIDER = 'Wharton Research Data Services (WRDS)'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/mergent-fixed-income-securities-database-fisd/'
LIBRARY = 'fisd'
TABLE = 'fisd_mergedissue'
ISSUER_TABLE = 'fisd_mergedissuer'
FREQ = 'D'
MIN_YEAR = 1894
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'complete_cusip'
ENTITY_ID_IN_CLEAN_DSET = 'cusip'
TIME_VAR_IN_RAW_DSET = 'offering_date'
TIME_VAR_IN_CLEAN_DSET = 'offering_date'

In [ ]:
#| export
def list_all_vars() -> pd.DataFrame:
    "Collects names of all available variables from WRDS f`{LIBRARY}.{TABLE}`."

    try:
        db = wrds_api.Connection()
        issues = db.describe_table(LIBRARY,TABLE).assign(wrds_library=LIBRARY, wrds_table=TABLE)
        issuers = db.describe_table(LIBRARY,ISSUER_TABLE).assign(wrds_library=LIBRARY, wrds_table=ISSUER_TABLE)
    finally:
        db.close()

    return pd.concat([issues, issuers])[['name','type','wrds_library','wrds_table']].copy()

In [ ]:
#| eval: false
all_vars = list_all_vars()

In [ ]:
#| eval: false
all_vars

,name,type,wrds_library,wrds_table
0,issue_id,DOUBLE_PRECISION,fisd,fisd_mergedissue
1,issuer_id,DOUBLE_PRECISION,fisd,fisd_mergedissue
2,prospectus_issuer_name,VARCHAR(64),fisd,fisd_mergedissue
3,issuer_cusip,VARCHAR(6),fisd,fisd_mergedissue
4,issue_cusip,VARCHAR(3),fisd,fisd_mergedissue
...,...,...,...,...
17,country,VARCHAR(16),fisd,fisd_mergedissuer
18,main_phone,VARCHAR(20),fisd,fisd_mergedissuer
19,main_fax,VARCHAR(20),fisd,fisd_mergedissuer
20,note,VARCHAR(254),fisd,fisd_mergedissuer


In [ ]:
#| export
def default_raw_vars():
    """Defines default variables used in `get_raw_data` if none are specified."""

    return ['offering_date', 'issue_id', 'issuer_id', 'issuer_cusip', 'issue_cusip', 'complete_cusip', 'isin', 
            'security_level', 'coupon_type', 'convertible', 'foreign_currency', 'rule_144a', 'redeemable', 'bond_type', 
            'maturity', 'coupon', 'offering_amt', 'offering_price', 'principal_amt', 'defaulted',
            'day_count_basis', 'last_interest_date', 'first_interest_date', 'conv_commod_type',
            'cusip_name','naics_code','sic_code', 'treasury_maturity', 
            'country_domicile',  'private_placement', 'asset_backed', 'interest_frequency','dated_date',
            ]            

In [ ]:
print(default_raw_vars())

['offering_date', 'issue_id', 'issuer_id', 'issuer_cusip', 'issue_cusip', 'complete_cusip', 'isin', 'security_level', 'coupon_type', 'convertible', 'foreign_currency', 'rule_144a', 'redeemable', 'bond_type', 'conv_commod_type', 'maturity', 'coupon', 'offering_amt', 'offering_price', 'principal_amt', 'defaulted', 'day_count_basis', 'last_interest_date', 'first_interest_date', 'cusip_name', 'naics_code', 'sic_code', 'treasury_maturity', 'country_domicile', 'private_placement', 'asset_backed', 'interest_frequency', 'dated_date']


In [ ]:
#| export
def parse_varlist(vars: List[str]=None,
                  required_vars = [],
                  ) -> str:
    """Validates that `vars` are available in `{LIBRARY}.{TABLE}` table and adds a. prefixes to variable names to feed into an SQL query"""

    # Get all available variables and add suffixes needed for the SQL query
    suffix_mapping = {TABLE: 'a.', ISSUER_TABLE: 'b.'}
    all_avail_vars = list_all_vars().drop_duplicates(subset='name',keep='first').copy()
    all_avail_vars['w_prefix'] = all_avail_vars.apply(lambda row: suffix_mapping[row['wrds_table']] + row['name'] , axis=1)

    if vars == '*': return ','.join(list(all_avail_vars['w_prefix']))
    
    # Add required vars to requested vars
    if vars is None: vars = default_raw_vars()
    vars_to_get =  required_vars + [x for x in list(set(vars)) if x not in required_vars]

    # Validate variables to be downloaded (make sure that they are in the target database)
    invalid_vars = [v for v in vars_to_get if v not in list(all_avail_vars.name)]
    if invalid_vars: raise ValueError(f"These vars are not in the database: {invalid_vars}") 

    # Extract information on which variable comes from which wrds table, so we know what prefix to use
    vars_to_get = pd.DataFrame(vars_to_get, columns=['name'])
    get_these = vars_to_get.merge(all_avail_vars, how = 'left', on = 'name')
        
    return ','.join(list(get_these['w_prefix']))

In [ ]:
#| eval: false
parse_varlist()

'a.foreign_currency,a.asset_backed,a.coupon,a.interest_frequency,a.isin,a.complete_cusip,a.principal_amt,a.defaulted,a.offering_price,a.private_placement,a.issuer_cusip,a.bond_type,a.offering_amt,a.treasury_maturity,a.dated_date,b.country_domicile,a.security_level,a.redeemable,b.naics_code,a.rule_144a,b.sic_code,b.cusip_name,a.issue_cusip,a.offering_date,a.convertible,a.first_interest_date,a.last_interest_date,a.maturity,a.issue_id,a.coupon_type,a.issuer_id,a.day_count_basis,a.conv_commod_type'

In [ ]:
#| export
def get_raw_data(
        vars: List[str]=None, # If None, downloads `default_raw_vars`; use '*' to get all available variables
        required_vars: List[str]=['offering_date','complete_cusip','issuer_id'], #Will get downloaded, even if not in `vars`
        nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
        start_date: str=None, # Start date in MM/DD/YYYY format
        end_date: str=None #End date in MM/DD/YYYY format
) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS `{LIBRARY}.{TABLE}` library"""
 
    wrds_api.validate_dates([start_date, end_date])
    vars = parse_varlist(vars, required_vars=required_vars)

    sql_string=f"""SELECT {vars}
            TSLA        FROM {LIBRARY}.{TABLE} AS a
                    LEFT JOIN {LIBRARY}.{ISSUER_TABLE} AS b ON a.issuer_id = b.issuer_id
                    WHERE 1=1
                """
    if start_date is not None: sql_string += r" AND offering_date >= %(start_date)s"
    if end_date is not None: sql_string += r" AND offering_date <= %(end_date)s"
    if nrows is not None: sql_string += r" LIMIT %(nrows)s"
    
    return wrds_api.download(sql_string,
                             params={'start_date':start_date, 'end_date':end_date, 'nrows':nrows})

In [ ]:
#| eval: false
raw = get_raw_data(start_date='01/01/1970')

In [ ]:
#| eval: false
raw.head(0)

,offering_date,complete_cusip,issuer_id,foreign_currency,asset_backed,coupon,interest_frequency,isin,principal_amt,defaulted,...,cusip_name,issue_cusip,convertible,first_interest_date,last_interest_date,maturity,issue_id,coupon_type,day_count_basis,conv_commod_type


In [ ]:
#| export
def process_raw_data(
        df: pd.DataFrame=None,  # Must contain `permno` and `datadate` columns   
) -> pd.DataFrame:
    """Mainly converts variables to categorical type to save memory."""

    df = df.rename(columns={'complete_cusip':'cusip'})
    to_cat = ['cusip', 'issuer_cusip', 'issue_cusip', 'isin', 'naics_code', 'sic_code', 
           'redeemable', 'security_level', 'country_domicile', 'private_placement', 'foreign_currency', 'rule_144a',
           'asset_backed', 'convertible', 'coupon_type','bond_type']
    for col in to_cat:
        if col in df.columns:
            df[col] = df[col].astype('string').astype('category')

    return df 

In [ ]:
#| eval: false
df_clean = process_raw_data(raw)

In [ ]:
#| eval: false
df_clean.head(0)

,offering_date,cusip,issuer_id,foreign_currency,asset_backed,coupon,interest_frequency,isin,principal_amt,defaulted,...,cusip_name,issue_cusip,convertible,first_interest_date,last_interest_date,maturity,issue_id,coupon_type,day_count_basis,conv_commod_type


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()